<a href="https://colab.research.google.com/github/hgbrian/Bayesian-Linear-Regression-Experiments/blob/master/simple_bayesian_analysis_inference_of_coronavirus_infection_rate_from_the_stanford_study_in_santa_clara_county.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pystan

In [0]:
import pystan
import io

In [0]:
model1 = """data {
  int y_sample;
  int n_sample;
  int y_spec;
  int n_spec;
  int y_sens;
  int n_sens;
}
parameters {
  real<lower = 0, upper = 1> p;
  real<lower = 0, upper = 1> spec;
  real<lower = 0, upper = 1> sens;
}
model {
  real p_sample;
  p_sample = p * sens + (1 - p) * (1 - spec);
  y_sample ~ binomial(n_sample, p_sample);
  y_spec ~ binomial(n_spec, spec);
  y_sens ~ binomial(n_sens, sens);
}"""

In [15]:
sm1 = pystan.StanModel(io.StringIO(model1))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6f756a2a99ec06dc98376140b81b067e NOW.


In [19]:
data1a = {"y_sample":50, "n_sample":3330, "y_spec":369+30, "n_spec":371+30, "y_sens":25+78, "n_sens":37+85}
fit1a = sm1.sampling(data=data1a, iter=10000, chains=4, warmup=500, thin=1, seed=101);
fit1a

Inference for Stan model: anon_model_6f756a2a99ec06dc98376140b81b067e.
4 chains, each with iter=10000; warmup=500; thin=1; 
post-warmup draws per chain=9500, total post-warmup draws=38000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
p      0.01  3.8e-5 4.5e-3 1.4e-3 7.2e-3   0.01   0.01   0.02  14111    1.0
spec   0.99  2.9e-5 3.3e-3   0.99   0.99   0.99    1.0    1.0  13446    1.0
sens   0.84  2.3e-4   0.03   0.77   0.82   0.84   0.86    0.9  19642    1.0
lp__ -338.2    0.01    1.3 -341.5 -338.7 -337.8 -337.2 -336.7  10319    1.0

Samples were drawn using NUTS at Sat May  2 14:39:24 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

In [21]:
data1b = {"y_sample":50, "n_sample":3330, "y_spec":3308, "n_spec":3324, "y_sens":130, "n_sens":157}
fit1b = sm1.sampling(data=data1b, iter=10000, chains=4, warmup=500, thin=1, seed=101);
fit1b

Inference for Stan model: anon_model_6f756a2a99ec06dc98376140b81b067e.
4 chains, each with iter=10000; warmup=500; thin=1; 
post-warmup draws per chain=9500, total post-warmup draws=38000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
p      0.01  1.9e-5 3.0e-3 6.7e-3   0.01   0.01   0.01   0.02  25530    1.0
spec   0.99  8.1e-6 1.2e-3   0.99   0.99   0.99    1.0    1.0  23300    1.0
sens   0.82  1.9e-4   0.03   0.76    0.8   0.82   0.84   0.88  26257    1.0
lp__ -446.1 10.0e-3   1.26 -449.4 -446.7 -445.8 -445.2 -444.7  15926    1.0

Samples were drawn using NUTS at Sat May  2 14:39:34 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

In [0]:
model2a = """data {
  int y_sample;
  int n_sample;
  int J_spec;
  int y_spec [J_spec];
  int n_spec [J_spec];
  int J_sens;
  int y_sens [J_sens];
  int n_sens [J_sens];
}
parameters {
  real p;
  vector[J_spec] e_spec;
  vector[J_sens] e_sens;
  real mu_spec;
  real<lower = 0> sigma_spec;
  real mu_sens;
  real<lower = 0> sigma_sens;
}
transformed parameters {
  vector[J_spec] spec;
  vector[J_sens] sens;
  spec = inv_logit(mu_spec + sigma_spec * e_spec);
  sens = inv_logit(mu_sens + sigma_sens * e_sens);
}
model {
  real p_sample;
  p_sample = p * sens[1] + (1 - p) * (1 - spec[1]);
  y_sample ~ binomial(n_sample, p_sample);
  y_spec ~ binomial(n_spec, spec);
  y_sens ~ binomial(n_sens, sens);
  e_spec ~ normal(0, 1);
  e_sens ~ normal(0, 1);
  sigma_spec ~ normal(0, 1);
  sigma_sens ~ normal(0, 1);
}"""

In [0]:
model2b = """data {
  int y_sample;  // for j = 1
  int n_sample;  // for j = 1
  int J_spec;
  int y_spec [J_spec];  // no samples for j > 1
  int n_spec [J_spec];
  int J_sens;
  int y_sens [J_sens];  // no samples for j > 1
  int n_sens [J_sens];
}
parameters {
  real p;
  real mu_logit_spec;
  real<lower = 0> sigma_logit_spec;
  real mu_logit_sens;
  real<lower = 0> sigma_logit_sens;
  vector<offset = mu_logit_spec, multiplier = sigma_logit_spec>[J_spec] logit_spec;
  vector<offset = mu_logit_sens, multiplier = sigma_logit_sens>[J_sens] logit_sens;
}
transformed parameters {
  vector<lower = 0, upper = 1>[J_spec] spec = inv_logit(logit_spec);
  vector<lower = 0, upper = 1>[J_sens] sens = inv_logit(logit_sens);
}
model {
  real p_sample = p * sens[1] + (1 - p) * (1 - spec[1]);  // j = 1
  y_sample ~ binomial(n_sample, p_sample);                // j = 1
  y_spec ~ binomial_logit(n_spec, logit_spec);
  y_sens ~ binomial_logit(n_sens, logit_sens);
  logit_spec ~ normal(mu_logit_spec, sigma_logit_spec);
  logit_sens ~ normal(mu_logit_sens, sigma_logit_sens);
  sigma_logit_spec ~ normal(0, 1);
  sigma_logit_sens ~ normal(0, 1);
}"""

In [32]:
sm2a = pystan.StanModel(io.StringIO(model2a))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d0b622394f864e098022ebefc9a0a2ae NOW.


In [33]:
sm2b = pystan.StanModel(io.StringIO(model2b))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cce675f27ed74ed4cfdb5b12f94e12cd NOW.


In [34]:
data2 = {"y_sample":50, "n_sample":3330, "J_spec":14, "y_spec":[0, 368, 30, 70, 1102, 300, 311, 500, 198, 99, 29, 146, 105, 50], 
         "n_spec":[0, 371, 30, 70, 1102, 300, 311, 500, 200, 99, 31, 150, 108, 52], "J_sens":4, 
         "y_sens":[0, 78, 27, 25], "n_sens":[0, 85, 37, 35]}
fit2a = sm2a.sampling(data=data2, iter=10000, chains=4, warmup=500, thin=1, seed=101);
fit2a

Inference for Stan model: anon_model_d0b622394f864e098022ebefc9a0a2ae.
4 chains, each with iter=10000; warmup=500; thin=1; 
post-warmup draws per chain=9500, total post-warmup draws=38000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
p            0.48    0.24   4.63  -6.19 7.7e-3   0.02   0.28   11.9    377   1.01
e_spec[1]    0.04    0.05   0.87  -1.48   -0.6  -0.03   0.62   1.92    276   1.02
e_spec[2]   -0.38    0.02   0.48  -1.34  -0.71   -0.4  -0.06   0.61    765   1.01
e_spec[3]    0.34    0.04   0.86  -1.26  -0.24    0.3   0.88   2.12    440   1.01
e_spec[4]    0.38    0.04    0.8  -1.06  -0.17   0.32   0.89   2.05    435   1.01
e_spec[5]    1.27    0.02   0.67 1.2e-3   0.79   1.24   1.71   2.67    833   1.01
e_spec[6]     0.8    0.04   0.73   -0.5   0.28   0.76   1.29   2.32    358   1.01
e_spec[7]    0.77    0.02    0.7  -0.48   0.27   0.74    1.2   2.33   1117   1.01
e_spec[8]    0.97    0.03   0.72  -0.35   0.46   0.92   1.45   2.45    5

In [35]:
fit2b = sm2b.sampling(data=data2, iter=10000, chains=4, warmup=500, thin=1, seed=101);
fit2b

Inference for Stan model: anon_model_cce675f27ed74ed4cfdb5b12f94e12cd.
4 chains, each with iter=10000; warmup=500; thin=1; 
post-warmup draws per chain=9500, total post-warmup draws=38000.

                   mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
p                  4.08    2.95  18.59 -14.79 8.8e-3   0.02   1.11  82.19     40   1.12
mu_logit_spec      5.67    0.01   0.66   4.53   5.23   5.61   6.05   7.09   2143    1.0
sigma_logit_spec   1.69  9.9e-3   0.47    0.9   1.35   1.64   1.98   2.74   2265    1.0
mu_logit_sens      0.53    0.54   1.24  -2.19  -0.33   0.75   1.46   2.47      5   1.28
sigma_logit_sens   1.63    0.43    0.8   0.29   0.93   1.69   2.21   3.13      3   1.53
logit_spec[1]      5.81    0.04   1.65    3.1   4.59   5.69   6.79   9.62   1436    1.0
logit_spec[2]      5.06  8.7e-3    0.6   4.02   4.64   5.02   5.41   6.37   4661    1.0
logit_spec[3]      6.11    0.03   1.61   3.48   4.99   5.95   7.03   9.86   2725    1.0
logit_spec[4]     